## FashionMNIST Dataset

- **FashionMNIST** is a dataset of Zalando article images.
- It contains **70,000 images** of size 28x28 pixels.
- The dataset has **10 classes**, each representing a different clothing item:
  1. T-shirt/top
  2. Trouser
  3. Pullover
  4. Dress
  5. Coat
  6. Sandal
  7. Shirt
  8. Sneaker
  9. Bag
  10. Ankle boot

In [ ]:
import matplotlib.pyplot as plt
import torch.optim

from torch.utils.data import random_split, DataLoader
from torchvision import transforms, datasets

from src.autoencoder import AutoEncoder
from src.hyper_parameters import HyperParameters

## Initialize hyper-parameters

In [ ]:
hyper_params = HyperParameters(
    learning_rate=0.001,
    batch_size=128,
    epochs=100,
)

## Initialization dataset to transformation:

- Convert to Grayscale (3 channels)
- Convert to Tensor (convert scale from 0-255 to 0.0-1.0)

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

## Dataset split and DataLoader preparation
- Calculate the total dataset size: dataset_size = len(dataset)
- Split the dataset into:
  - train_dataset – 70% of the data
  - val_dataset – 15% of the data
  - test_dataset – remaining 15% of the data
- Create DataLoaders for each split:
  - batch_size=32
  - shuffle=True for the training set
  - num_workers=4 and pin_memory=True for faster data loading
- Print dataset information:
  - number of samples in each split
  - number of classes
  - list of class names


In [ ]:
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=hyper_params.batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=hyper_params.batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=hyper_params.batch_size, shuffle=False, num_workers=4, pin_memory=True)

classes_str = ", ".join(f"{cls}" for cls in dataset.classes)
print(f"Training samples: {len(train_dataset)} (70%)")
print(f"Validation samples: {len(val_dataset)} (15%)")
print(f"Test samples: {len(test_dataset)} (15%)")
print(f"Number of classes: {len(dataset.classes)}")
print(f"Classes: {classes_str}")

## Display sample images from the training set

- Retrieve a batch of images and labels from the training DataLoader
- Display 10 sample images in a 2x5 grid
- Rearrange tensor from (C, H, W) to (H, W, C) for proper visualization
- Show corresponding class names as titles
- Hide axes for a cleaner look

In [ ]:
examples, labels = next(iter(train_loader))
fig, axes = plt.subplots(2, 5, figsize=(8, 3))
for i, ax in enumerate(axes.flat):
    img = examples[i].permute(1, 2, 0)
    ax.imshow(img)
    ax.set_title(dataset.classes[labels[i]])
    ax.axis("off")
plt.tight_layout()
plt.show()

## Initialize autoencoder

In [ ]:
model = AutoEncoder()

## Set an optimizer

In [ ]:
criterion = torch.optim.Adam(
    lr=hyper_params.learning_rate,
    params=model.parameters(),
    weight_decay=1e-8,
)